# Variogram modeling

In this tutorial, we illustrate one possible workflow for fitting variograms with GeoStats.jl.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "GeoStatsImages", "Plots", "GR", "Interact"]
    Pkg.add(pkg)
end
# make sure this tutorial is reproducible
srand(2017);

INFO: Package GeoStats is already installed
INFO: Package GeoStatsImages is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed
INFO: Package Interact is already installed


## The data

Let's consider a simple 2D problem in which properties of a field are sampled at random locations. We will be using an image of the Walker Lake in Nevada available in `GeoStatsImages.jl` as our field:

In [2]:
using GeoStatsImages
using Plots; gr()

field = training_image("WalkerLake")[:,:,1]

heatmap(field, label="Walker Lake, NV")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
<image width="18" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFsCAYAAADBkelSAAACHUlEQVR4nO2c223EMAwEqUD9V5Eu
4yOvBPpjEAwIqoCF9kU5vljnyd8KYP0QIBERN+thgAoC4qhVzqVmcw0Te7BGHLWgkk0BgdQo1yZr
lH8Q0Ca7Wxy1Ux8GKBICOhAQaT+lESQ2aD9Gbe1vgU4mAiSkRgEpk824NjnZRvsh1yj7la6N1Qi0
35ZsnWtC+yORF1pxo3zUMNcYjZTUVqMeCMFBqUFAUGeN1HRA5Ds2CKjyIEBK+xlqt8qnEUSNAiLf
+Y+lFslgTdaImkc3itmUkJpxZm+ym3XLppHwOKI0Eh5HXPt1gw2035Zs0P6x7QeT7aO2rrVAtooY
2++z33aukf8PKSutUKNN9j8C5WrUraXWL18gZ2s09mlk8DzaZPdAQvtDRk1oP/TjupMaZT+Uo8Gl
NdpPAQmpYa7pks2VFhtsOo249iMwS+3NMs5sBEbZ/qXWA/ke2AfPbIya8WEUAtJVZO3vF2g/g6Oc
2XM1AqnZkg0eRxAQtSOlRmt/s4TUVqMXQAyOcmbr5pFu1N6CPqoRJntwaXU5MrZfl+zB1MCfM6Dv
sn3n2s7sF0A6apTYGDWhRkmVVpjswa5R1Ar6etmo0cGo2a4JoXYE2n909nPJttkPltZ2/xGYbOge
LZ/9Oo2WWr/A0kI34GA7Iv/KnksNm0eDNeKo6dq/rvVAVEWoHQmHf+kumcaoBfQmArwngnON0sjX
fs7+udQgoC/rNOfnsknqIwAAAABJRU5ErkJggg==
" transform="translate(532, 12)"/>
 
 
 0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
<polyline clip-path="url(#clip4700)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 550.315,375.813 550.315,375.813 556.315,375.813 550.315,375.813 550.315,339.413 556.315,339.413 550.315,339.413 550.315,303.013 556.315,303.013 550.315,303.013 
 550.315,266.613 556.315,266.613 550.315,266.613 550.315,230.212 556.315,230.212 550.315,230.212 550.315,193.812 556.315,193.812 550.315,193.812 550.315,157.412 
 556.315,157.412 550.315,157.412 550.315,121.012 556.315,121.012 550.315,121.012 550.315,84.6115 556.315,84.6115 550.315,84.6115 550.315,48.2113 556.315,48.2113 
 550.315,48.2113 550.315,11.811 556.315,11.811 550.315,11.811 
 "/>

We sample a thousand points from the image and plot them:

In [3]:
# sample points
nx, ny = size(field)
idxs = rand(1:nx*ny, 1000)
vals = field[idxs]

# reshape arrays
X = ind2sub((nx,ny), idxs)
X = [X[1] X[2]]'
X = Float64.(X)
z = vals

# plot (y,x) to match image orientation
scatter(X[2,:], X[1,:], c=vals)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<circle clip-path="url(#clip4902)" style="fill:#000000; stroke:none

## Empirical variogram

We consider a simple omnidirectional variogram. Variogram objects in GeoStats.jl are integrated with the Julia Plots.jl framework. We can plot them directly and interact with the result as we vary the number of bins used for estimation:

In [4]:
using GeoStats
using Interact

@manipulate for nbins=1:30
    γ = EmpiricalVariogram(X, z, nbins=nbins)
    
    plot(γ)
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 15 Any , "nbins", 15, "15", 15, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(18, "18"),Pair{Any,Any}(30, "30"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(16, "16"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(21, "21"),Pair{Any,Any}(7, "7"),Pair{Any,Any}(9, "9"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(10, "10")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram

Besides the variogram itself, GeoStats.jl presents the bin counts (scaled) as a measure of confidence about the estimated points. This frequency plot can be deactived by passing the option `bincounts=false` to the plot command. We encourage users to keep the bin counts option activated as it has zero cost.

This empirical variogram was constructed using the Euclidean distance between data locations. We can also specify a custom distance to estimate the variogram when points are embedded on different coordinate systems. Please consult the documentation for more distance functions.

After interacting with the plot, we select a number of bins and proceed to fitting a theoretical model:

In [5]:
# final empirical variogram
γemp = EmpiricalVariogram(X, z, nbins=17)

plot(γemp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram

## Theoretical variogram

We will exploit the Interact.jl package again to fit a theoretical variogram to the data. Various theoretical variogram models are available in GeoStats.jl, including a composite additive model that can be used to combine different variogram types. Please consult the documentation for more details. Here we will use a simple spherical variogram:

In [6]:
@manipulate for s=linspace(0,.1), r=linspace(0,100), n=linspace(0,.1)
    # theoretical variogram
    γtheo = SphericalVariogram(sill=s, range=r, nugget=n)

    plot(γemp)
    plot!(γtheo, maxlag=500., label="theoretical")
end

Interact.Options{:SelectionSlider,Any}(5: "input-2" = 0.04897959183673469 Any , "s", 0.04897959183673469, "0.04897959183673469", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict{Any,Any}(Pair{Any,Any}(0.0102041, "0.01020408163265306"),Pair{Any,Any}(0.0183673, "0.018367346938775512"),Pair{Any,Any}(0.0265306, "0.026530612244897958"),Pair{Any,Any}(0.0877551, "0.08775510204081632"),Pair{Any,Any}(0.0163265, "0.0163265306122449"),Pair{Any,Any}(0.0693878, "0.06938775510204082"),Pair{Any,Any}(0.0857143, "0.08571428571428572"),Pair{Any,Any}(0.0306122, "0.030612244897959183"),Pair{Any,Any}(0.0387755, "0.03877551020408163"),Pair{Any,Any}(0.0367347, "0.036734693877551024")…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Any}(7: "input-3" = 48.97959183673469 Any , "r", 48.97959183673469, "48.97959183673469", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.0"=>0.0,"2.0408163265306123"=>2.04082,"4.081632653061225"=>4.08163,"6.122448979591836"=>6.12245,"8.16326530612245"=>8.16327,"10.204081632653061"=>10.2041,"12.244897959183673"=>12.2449,"14.285714285714286"=>14.2857,"16.3265306122449"=>16.3265,"18.367346938775512"=>18.3673…), Dict{Any,Any}(Pair{Any,Any}(63.2653, "63.265306122448976"),Pair{Any,Any}(75.5102, "75.51020408163265"),Pair{Any,Any}(46.9388, "46.93877551020408"),Pair{Any,Any}(22.449, "22.448979591836736"),Pair{Any,Any}(55.102, "55.10204081632653"),Pair{Any,Any}(59.1837, "59.183673469387756"),Pair{Any,Any}(95.9184, "95.91836734693878"),Pair{Any,Any}(93.8776, "93.87755102040816"),Pair{Any,Any}(71.4286, "71.42857142857143"),Pair{Any,Any}(18.3673, "18.367346938775512")…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Any}(9: "input-4" = 0.04897959183673469 Any , "n", 0.04897959183673469, "0.04897959183673469", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict{Any,Any}(Pair{Any,Any}(0.0102041, "0.01020408163265306"),Pair{Any,Any}(0.0183673, "0.018367346938775512"),Pair{Any,Any}(0.0265306, "0.026530612244897958"),Pair{Any,Any}(0.0877551, "0.08775510204081632"),Pair{Any,Any}(0.0163265, "0.0163265306122449"),Pair{Any,Any}(0.0693878, "0.06938775510204082"),Pair{Any,Any}(0.0857143, "0.08571428571428572"),Pair{Any,Any}(0.0306122, "0.030612244897959183"),Pair{Any,Any}(0.0387755, "0.03877551020408163"),Pair{Any,Any}(0.0367347, "0.036734693877551024")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram 
 
 
 
 theoretical

After tuning the parameters of the theoretical variogram interactively, we plot the final result:

In [7]:
γtheo = SphericalVariogram(sill=.083, range=32.653)

plot(γemp, label="empirical")
plot!(γtheo, maxlag=500., label="theoretical")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 empirical 
 
 
 
 theoretical

## Empirical cross-variogram

For empirical cross-variograms, we pass two observation vectors instead of one:

In [8]:
z₁ = z
z₂ = z + randn()

@manipulate for nbins=1:30
    γ = EmpiricalVariogram(X, z₁, z₂, nbins=nbins)
    
    plot(γ)
end

Interact.Options{:SelectionSlider,Any}(13: "input-5" = 15 Any , "nbins", 15, "15", 15, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(18, "18"),Pair{Any,Any}(30, "30"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(16, "16"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(21, "21"),Pair{Any,Any}(7, "7"),Pair{Any,Any}(9, "9"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(10, "10")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 Lag h 
 
 
 Variogram(h) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 bin counts 
 
 
 
 
 variogram